In [5]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Add, Input
from tensorflow.keras.models import Model
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [6]:
train_file_id = '1JcAvFAmCrYtlpYUFiR3gTUX5CizmjoJv'
test_file_id = '1UMe6kjf4SEuHzxSmrg-ZyKA-ZOWU5WOz'

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded_train = drive.CreateFile({'id': train_file_id})
downloaded_train.GetContentFile('train.csv')
downloaded_test = drive.CreateFile({'id': test_file_id})
downloaded_test.GetContentFile('test.csv')

df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [8]:
# x = df.drop('labels',axis = 1)
# y = df['labels']
x_train = df.drop('labels', axis=1)
y_train = df['labels']
x_id = df2['id']
x_test = df2.drop('id', axis=1)

In [9]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
input_shape = (1200, 1)
num_classes = 2

cnn_inputs = Input(shape=input_shape)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Dense(units=2400, activation='relu')(x)
x = Dense(units=1200, activation='relu')(x)
x = Dense(units=600, activation='relu')(x)
x = Dense(units=300, activation='relu')(x)
x = Flatten()(x)
cnn_outputs = Dense(units=300, activation='relu')(x)

def residual_block(inputs, filters, kernel_size):
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(inputs)
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = Add()([x, inputs])
    x = MaxPooling1D(pool_size=2)(x)
    return x

residual_inputs = Reshape((-1, 1))(cnn_outputs)
x = residual_block(residual_inputs, filters=50, kernel_size=4)
x = Conv1D(filters=50, kernel_size=4, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=3)(x)
x = residual_block(x, filters=50, kernel_size=4)
x = residual_block(x, filters=50, kernel_size=4)
x = residual_block(x, filters=50, kernel_size=4)
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)
x = Dense(units=10, activation='relu')(x)
outputs = Dense(units=1, activation='sigmoid')(x)


In [ ]:
model = Model(inputs=cnn_inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=25)

Epoch 1/25
  1/165 [..............................] - ETA: 45:00 - loss: 0.6929 - accuracy: 0.4688

In [ ]:
predictions = model.predict(x_test)
predicted_labels = (predictions > 0.5).astype(int)
result_df = pd.DataFrame({'id': x_id, 'labels': predicted_labels.flatten()})
result_df.to_csv('predictions.csv', index=False)
from google.colab import files
files.download('predictions.csv')